#### 특정 시장의 업종별 수익률 상위 n개 종목의 위험도 분포를 파이차트로 시각화

In [1]:
# 데이터 분석을 위한 패키지
import numpy as np
import pandas as pd

In [2]:
# 시각화 패키지
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [3]:
import FinanceDataReader as fdr

In [4]:
import koreanize_matplotlib

In [5]:
from data_loader import load_sector_data

In [6]:
# 모든 컬럼,행을 출력하도록 설정
pd.set_option('display.max_columns', None) # None으로 설정하면 모든 컬럼 출력
pd.set_option('display.max_rows', None) #None으로 설정하면 모든 행 출력

In [7]:
# 각 시장별 업종 리스트
def print_sector_lists(market):
  
  if market in ['KOSPI', 'KOSDAQ', 'ETF'] : 
    sector_list = load_sector_data(market=market)['Sector'].unique()
  else : 
    raise ValueError('KOSPI, KOSDAQ, ETF 중 하나의 시장을 입력해야 합니다.')
  
  print(f'{market} 시장의 업종 리스트 : {sector_list}')
    

In [8]:
# 특정 시장의 업종별 수익률 상위 n개 종목의 위험도 분포를 파이차트로 시각화
def visualize_sector_risk_distribution(market=None, sector=None, month_ago=1, cnt=None):

    df = load_sector_data(market, month_ago=month_ago)
    if sector is not None:
        if sector not in df['Sector'].unique():
            raise ValueError(f"'{sector}'는(은) 유효한 섹터가 아닙니다. 사용 가능한 섹터: {df['Sector'].unique()}")
        df = df[df['Sector'] == sector]
    
    if cnt is not None :
        df = df.sort_values(by='TotalReturn', ascending=False).head(cnt)
    
    # 위험도별 종목 비중 계산
    risk_distribution = df['Risk'].value_counts().reset_index()
    risk_distribution.columns = ['Risk', 'Count']
    
    if sector is not None :
        if cnt is not None:   
            title = f'{market}시장 {sector}업종 내 {month_ago}개월 평균 수익률 상위 {cnt}개 종목 위험도 비중'
        else :
            title = f'{market}시장 {sector}업종 내 {month_ago}개월 종목 위험도 비중'
    elif cnt is not None:
        title = f'{market}시장 내 {month_ago}개월 수익률 상위 {cnt}개 종목 위험도 비중'
    else :
        title = f'{market}시장 내 {month_ago}개월 종목 위험도 비중'

    fig = px.pie(
        risk_distribution,
        values='Count',
        names='Risk',
        title=title,
        labels={'Risk': '위험도', 'Count': '종목 수'}
    )
    # 위험도 설명 추가
    risk_explanations = """
    위험도 설명:
    1: 낮음 - 변동성이 작고 안정적인 종목 (시장내 변동성 하위 25%)
    2: 중간 - 평균적인 변동성을 가진 종목 (시장내 변동성 25%~75%구간)
    3: 높음 - 변동성이 크고 위험한 종목   (시장내 변동성 75%이상 구간)
    """
    fig.add_annotation(
        text=risk_explanations,
        showarrow=False,
        xref="paper", yref="paper",
        x=0.5, y=-0.3,  # 위치 조정
        font=dict(size=12),  # 글자 크기
        align="center"
    )
    fig.update_layout(
        title_x=0.5,
        title_font=dict(size=20, color='black', family='Arial')
    ) 
   # 그래프 보여주기
    fig.show()

In [12]:
visualize_sector_risk_distribution('KOSDAQ', cnt=100)

1개월 간의 data를 불러옵니다.
